In [2]:
import os
import sys
import geopandas as gpd
import pandas as pd 


sys.path.append("../")
from config import base_path

In [54]:
path_to_geostat_2006 = os.path.join(base_path, "data/raw/GEOSTAT_Population/GEOSTAT_Grid_POP_2006_1K/GEOSTAT_grid_EU_POP_2006_1K_V1_1_1.csv")
path_to_geostat_2011 = os.path.join(base_path, "data/raw/GEOSTAT_Population/GEOSTAT-grid-POP-1K-2011-V2-0-1/Version 2_0_1/GEOSTAT_grid_POP_1K_2011_V2_0_1.csv")
path_to_geostat_2018 = os.path.join(base_path, "data/raw/GEOSTAT_POPULATION/JRC_GRID_2018/JRC_POPULATION_2018.shp")
path_to_geostat_2021 = os.path.join(base_path, "data/raw/GEOSTAT_Population/Eurostat_Census-GRID_2021_V1-0/ESTAT_Census_2011_V1-0.gpkg")
path_to_nuts_data = os.path.join(base_path, "data/raw/EU_NUTS/NUTS_RG_01M_2021_3035.shp")


In [17]:
# read in csv data from 2006 and 2011
geostat_2006 = pd.read_csv(path_to_geostat_2006, sep=";")
geostat_2011 = pd.read_csv(path_to_geostat_2011)


C:\Users\David\AppData\Local\Temp;\ipykernel_3704\2126506555.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  geostat_2011 = pd.read_csv(path_to_geostat_2011)


In [ ]:

# read in shp-file and geopackage from 2018 and 2021
geostat_2018 = gpd.read_file(path_to_geostat_2018)
geostat_2021 = gpd.read_file(path_to_geostat_2021)

In [55]:
# read in nuts data
nuts = gpd.read_file(path_to_nuts_data)

In [56]:
nuts.head()

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
0,CZ,0,CZ,Česko,Česko,0.0,0,0,CZ,"POLYGON ((4635755.041 3113254.655, 4636313.459..."
1,DE,0,DE,Deutschland,Deutschland,0.0,0,0,DE,"MULTIPOLYGON (((4355225.365 2715902.993, 43541..."
2,DK,0,DK,Danmark,Danmark,0.0,0,0,DK,"MULTIPOLYGON (((4650502.736 3591342.844, 46503..."
3,AL,0,AL,Shqipëria,Shqipëria,0.0,0,0,AL,"MULTIPOLYGON (((5129579.170 2204098.752, 51298..."
4,CY,0,CY,Kýpros,Κύπρος,0.0,0,0,CY,"MULTIPOLYGON (((6527040.718 1762367.593, 65267..."


In [82]:
# intersect geostat_2021 data with nuts austria level 0 shape
nuts_austria = nuts[(nuts.LEVL_CODE == 0) & (nuts.CNTR_CODE == "AT")]
nuts_austria.set_crs(epsg="3035", allow_override=True, inplace=True)
geostat_2021_austria = geostat_2021[geostat_2021.within(nuts_austria.unary_union)]

c:\Users\David\anaconda3\envs\pymc_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [84]:
geostat_2021_austria.head()

,GRD_ID,POP_2021,geometry
1630265,CRS3035RES1000mN2596000E4672000,0.0,"POLYGON ((4672000.000 2596000.000, 4673000.000..."
1632555,CRS3035RES1000mN2597000E4672000,0.0,"POLYGON ((4672000.000 2597000.000, 4673000.000..."
1634847,CRS3035RES1000mN2598000E4671000,0.0,"POLYGON ((4671000.000 2598000.000, 4672000.000..."
1634848,CRS3035RES1000mN2598000E4672000,0.0,"POLYGON ((4672000.000 2598000.000, 4673000.000..."
1637143,CRS3035RES1000mN2599000E4671000,1.0,"POLYGON ((4671000.000 2599000.000, 4672000.000..."


In [85]:
# rename population columns into a uniform way 
geostat_2006.rename(columns={"POP_TOT": "POP_2006"}, inplace=True)
geostat_2011.rename(columns={"TOT_P": "POP_2011"}, inplace=True)
geostat_2018.rename(columns={"TOT_P_2018": "POP_2018"}, inplace=True)
geostat_2021_austria.rename(columns={"OBS_VALUE_T": "POP_2021"}, inplace=True)


C:\Users\David\AppData\Local\Temp;\ipykernel_3704\3377811848.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geostat_2021_austria.rename(columns={"OBS_VALUE_T": "POP_2021"}, inplace=True)


In [86]:
# rename GRD_ID columns of data from 2006 and 2011 so that it matches the column names from 2018 and 2021

def convert_grd_id(grd_id: str):
    grd_str = grd_id.split("km")[1]
    nrd_component = grd_str.split("E")[0]
    east_component = "E" + grd_str.split("E")[1]
    final_str = f"CRS3035RES1000m{nrd_component}000{east_component}000"
    return final_str
    
geostat_2006.GRD_ID = geostat_2006.GRD_ID.apply(lambda x: convert_grd_id(x))
geostat_2011.GRD_ID = geostat_2011.GRD_ID.apply(lambda x: convert_grd_id(x))

In [87]:
# merge data from different years
geostat_pop = geostat_2021_austria.merge(geostat_2018.loc[:, ["GRD_ID", "POP_2018"]], on="GRD_ID", how="left")
geostat_pop = geostat_pop.merge(geostat_2011.loc[:, ["GRD_ID", "POP_2011"]], on="GRD_ID", how="left")
geostat_pop = geostat_pop.merge(geostat_2006.loc[:, ["GRD_ID", "POP_2006"]], on="GRD_ID", how="left")

In [88]:
# fill NaN values with 0
geostat_pop["POP_2006"] = geostat_pop["POP_2006"].fillna(0)
geostat_pop["POP_2011"] = geostat_pop["POP_2011"].fillna(0)
geostat_pop["POP_2018"] = geostat_pop["POP_2018"].fillna(0)
geostat_pop["POP_2021"] = geostat_pop["POP_2021"].fillna(0)


In [89]:
# bring columns into the right order
geostat_pop = geostat_pop.loc[:, ["GRD_ID", "POP_2006", "POP_2011", "POP_2018", "POP_2021", "geometry"]]

In [91]:
geostat_pop.head(10)

,GRD_ID,POP_2006,POP_2011,POP_2018,POP_2021,geometry
0,CRS3035RES1000mN2596000E4672000,0.0,0.0,0.0,0.0,"POLYGON ((4672000.000 2596000.000, 4673000.000..."
1,CRS3035RES1000mN2597000E4672000,0.0,0.0,0.0,0.0,"POLYGON ((4672000.000 2597000.000, 4673000.000..."
2,CRS3035RES1000mN2598000E4671000,0.0,0.0,0.0,0.0,"POLYGON ((4671000.000 2598000.000, 4672000.000..."
3,CRS3035RES1000mN2598000E4672000,0.0,0.0,0.0,0.0,"POLYGON ((4672000.000 2598000.000, 4673000.000..."
4,CRS3035RES1000mN2599000E4671000,1.0,1.0,1.0,1.0,"POLYGON ((4671000.000 2599000.000, 4672000.000..."
5,CRS3035RES1000mN2599000E4672000,3.0,2.0,2.0,3.0,"POLYGON ((4672000.000 2599000.000, 4673000.000..."
6,CRS3035RES1000mN2600000E4665000,0.0,0.0,0.0,0.0,"POLYGON ((4665000.000 2600000.000, 4666000.000..."
7,CRS3035RES1000mN2600000E4670000,14.0,20.0,19.0,8.0,"POLYGON ((4670000.000 2600000.000, 4671000.000..."
8,CRS3035RES1000mN2600000E4671000,1.0,1.0,1.0,1.0,"POLYGON ((4671000.000 2600000.000, 4672000.000..."
9,CRS3035RES1000mN2600000E4672000,0.0,0.0,0.0,0.0,"POLYGON ((4672000.000 2600000.000, 4673000.000..."


In [92]:
# export GeoDataFrame to shp file
geostat_pop.to_file(os.path.join(base_path, "data/processed/population_data/population_data_all_years_vector/geostat_pop.shp"))